### Week 1. Titanic Classification with DistilBERT + XGBoost 과제

Week 1. 과제로는 `week1_lecture.ipynb` 코드 예제를 참고하여, `kaggle`에서 titanic 데이터를 받아 데이터 전처리를 진행한 후 학습해 자신의 모델을 만든 후, ipynb 파일과 결과 csv파일을

- ***hw1_학번_이름.ipynb***
- ***submission.csv***

실전데이터사이언스 스터디 repository week1 폴더에 커밋하시면 됩니다.!

https://github.com/a2ran/prac_ds

타이타닉 데이터 전처리 예시는 다음과 같습니다.

1. Filling out missing (NaN) Values
2. VIF으로 분산이 높은 column 제거 or 수정
3. train_test_split 으로 train/val 나누는 과정 수정
4. "bert-base-uncased" 이외 다른 BERT 모델을 사용해 Embedding하기
5. "xgboost" 이외 다른 머신러닝 알고리즘 사용

참고할만한 example은 다음과 같습니다.


https://github.com/minsuk-heo/kaggle-titanic/blob/master/titanic-solution.ipynb

`Optional` :

https://github.com/mrdbourke/your-first-kaggle-submission/blob/master/kaggle-titanic-dataset-example-submission-workflow.ipynb
https://github.com/agconti/kaggle-titanic/blob/master/Titanic.ipynb


# 기본 제공 코드

### 1. Kaggle에서 데이터 받아오기

In [ ]:
## pip install ~ : 패키지 다운로드
## -q : 로그 메세지 출력 X

!pip install -q kaggle

In [ ]:
## Kaggle에서 데이터를 받아오기 위해서는 Authenticator Token인 "kaggle.json"이 있어야 합니다.
## 자세한 내용은 영상을 참고하세요

from google.colab import drive
drive.mount("/content/drive")

!mkdir ~/.kaggle
## Drive에 kaggle.json을 업로드한 경로를 적으시면 됩니다. ex) (/content/drive/MyDrive/study_session/kaggle.json)
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Mounted at /content/drive


In [ ]:
# 공모전 이름
competition_name = "titanic"

# 공모전 다운로드 to local environment
! kaggle competitions download -c {competition_name}

# {competition_name}이름의 폴더에 zip 파일 압축해제
! unzip {competition_name + ".zip"} -d {competition_name}

# 드라이브 확인을 완료했으므로 드라이브 mount를 해제합니다.
drive.flush_and_unmount()

  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 54.7MB/s]
Archive:  titanic.zip
  inflating: titanic/gender_submission.csv  
  inflating: titanic/test.csv        
  inflating: titanic/train.csv       


### 2. 데이터 전처리

In [ ]:
## GPU 활성화
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import numpy as np
import pandas as pd

train = pd.read_csv('titanic/train.csv')
test = pd.read_csv('titanic/test.csv')

In [ ]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
X = train.drop(columns = ['PassengerId', 'Survived', 'Name', 'Ticket', 'Cabin', 'Embarked'], axis = 1)
y = train['Survived']

#### 나이 결측치 예상

In [ ]:
X1 = X.dropna(axis=0, inplace=False)

In [ ]:
X1

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,male,22.0,1,0,7.2500
1,1,female,38.0,1,0,71.2833
2,3,female,26.0,0,0,7.9250
3,1,female,35.0,1,0,53.1000
4,3,male,35.0,0,0,8.0500
...,...,...,...,...,...,...
885,3,female,39.0,0,5,29.1250
886,2,male,27.0,0,0,13.0000
887,1,female,19.0,0,0,30.0000
889,1,male,26.0,0,0,30.0000


In [ ]:
X1 = pd.get_dummies(X1, columns=['Sex'], prefix=['Sex'])

In [ ]:
X1['Age_1'] = X1['Age']

In [ ]:
X1 = X1.drop('Age', axis=1)

In [ ]:
X1

,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male,Age_1
0,3,1,0,7.2500,0,1,22.0
1,1,1,0,71.2833,1,0,38.0
2,3,0,0,7.9250,1,0,26.0
3,1,1,0,53.1000,1,0,35.0
4,3,0,0,8.0500,0,1,35.0
...,...,...,...,...,...,...,...
885,3,0,5,29.1250,1,0,39.0
886,2,0,0,13.0000,0,1,27.0
887,1,0,0,30.0000,1,0,19.0
889,1,0,0,30.0000,0,1,26.0


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame([variance_inflation_factor(X1.iloc[:,1:].values, i) for i in range(X1.iloc[:,:-1].shape[1])], index = X1.iloc[:,:-1].columns)
vif.reset_index(inplace = True)
vif.columns = ['Feature', 'VIFscore']
vif

,Feature,VIFscore
0,Pclass,1.272781
1,SibSp,1.268241
2,Parch,1.102493
3,Fare,3.301502
4,Sex_female,4.805763
5,Sex_male,1.148806


In [ ]:
X1_y = X1['Age_1']
X1_x = X1.drop('Age_1', axis = 1)

In [ ]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression().fit(X1_x, X1_y)

In [ ]:
#위의 결과값을 보기 좋게 출력해보자!
linear_beta = linear_model.coef_
linear_intercept = linear_model.intercept_

for i in range(len(linear_beta)):
  print(f"{X1_x.columns[i]} : {linear_beta[i]: .2f}")
print(f"Intercept(Beta0) :{linear_intercept: .2f}")

Pclass : -7.01
SibSp : -3.80
Parch : -0.77
Fare : -0.02
Sex_female : -1.62
Sex_male :  1.62
Intercept(Beta0) : 47.85


In [ ]:
X2 = X

In [ ]:
X2 = pd.get_dummies(X2, columns=['Sex'], prefix=['Sex'])

In [ ]:
X2

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male
0,3,22.0,1,0,7.2500,0,1
1,1,38.0,1,0,71.2833,1,0
2,3,26.0,0,0,7.9250,1,0
3,1,35.0,1,0,53.1000,1,0
4,3,35.0,0,0,8.0500,0,1
...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,0,1
887,1,19.0,0,0,30.0000,1,0
888,3,NaN,1,2,23.4500,1,0
889,1,26.0,0,0,30.0000,0,1


In [ ]:
# 주어진 회귀 결과
regression_coeffs = {
    'Pclass': -7.01,
    'SibSp': -3.80,
    'Parch': -0.77,
    'Fare': -0.02,
    'Sex_female': -1.62,
    'Sex_male': 1.62,
    'Intercept': 47.85
}

# 결측치 채우기
X2['Age'] = X2['Age'].fillna(
    regression_coeffs['Intercept'] +
    regression_coeffs['Pclass'] * X2['Pclass'] +
    regression_coeffs['SibSp'] * X2['SibSp'] +
    regression_coeffs['Parch'] * X2['Parch'] +
    regression_coeffs['Fare'] * X2['Fare'] +
    regression_coeffs['Sex_female'] * X2['Sex_female'] +
    regression_coeffs['Sex_male'] * X2['Sex_male']
)

In [ ]:
X2

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male
0,3,22.000,1,0,7.2500,0,1
1,1,38.000,1,0,71.2833,1,0
2,3,26.000,0,0,7.9250,1,0
3,1,35.000,1,0,53.1000,1,0
4,3,35.000,0,0,8.0500,0,1
...,...,...,...,...,...,...,...
886,2,27.000,0,0,13.0000,0,1
887,1,19.000,0,0,30.0000,1,0
888,3,19.391,1,2,23.4500,1,0
889,1,26.000,0,0,30.0000,0,1


In [ ]:
X['Age'] = X2['Age'].round().astype(int)

In [ ]:
X.isnull().sum()

Pclass    0
Sex       0
Age       0
SibSp     0
Parch     0
Fare      0
dtype: int64

In [ ]:
X

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,male,22,1,0,7.2500
1,1,female,38,1,0,71.2833
2,3,female,26,0,0,7.9250
3,1,female,35,1,0,53.1000
4,3,male,35,0,0,8.0500
...,...,...,...,...,...,...
886,2,male,27,0,0,13.0000
887,1,female,19,0,0,30.0000
888,3,female,19,1,2,23.4500
889,1,male,26,0,0,30.0000


#### train test split

In [ ]:
X

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,male,22,1,0,7.2500
1,1,female,38,1,0,71.2833
2,3,female,26,0,0,7.9250
3,1,female,35,1,0,53.1000
4,3,male,35,0,0,8.0500
...,...,...,...,...,...,...
886,2,male,27,0,0,13.0000
887,1,female,19,0,0,30.0000
888,3,female,19,1,2,23.4500
889,1,male,26,0,0,30.0000


In [ ]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 77, stratify = y)

In [ ]:
## Your Codes

### 3. 데이터 임베딩

In [ ]:
from typing import List
from tqdm.notebook import tqdm

!pip install -q transformers
from transformers import AutoModel, AutoTokenizer

class Encode_with_BERT:
    def __init__(self):
        ## Huggingface에서 BERT 모델을 받아옵니다.
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
        self.model = AutoModel.from_pretrained('bert-base-uncased').to(device)

    ## 입력한 `texts`을 768 길이의 숫자벡터로 특징을 추출합니다.
    def extract(self, texts: List[str]):
        features = np.zeros((len(texts), 768), dtype = np.float16)
        for index, text in enumerate(tqdm(texts)):
            tokenized_text = self.tokenizer(text, return_tensors="pt").to(device)
            model_output = self.model(**tokenized_text)[0].detach().cpu()
            features[index, :] = model_output.numpy().mean(axis=1)

        return features

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.2 MB/s eta 0:00:00


### 4. 임베딩한 데이터 머신러닝 알고리즘으로 분류

In [ ]:
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

extractor = Encode_with_BERT()
scaler = StandardScaler()
classifier = XGBClassifier(use_label_encoder = False)

## XGBoost 학습
train_labels = [int(_) for _ in y_train.values] ## [0,1,1, ...]
texts = [", ".join(str(_)) for _ in X_train.values] ##[[0,1,male],[2,1,female]...]
train_features = scaler.fit_transform(extractor.extract(texts)) ## to (len(texts), 768) 숫자벡터
classifier.fit(train_features, train_labels) ## XGBoost train

## Prediction with Test Data
answer = [int(_) for _ in y_val.values]
texts = [", ".join(str(_)) for _ in X_val.values]
preds = classifier.predict(scaler.transform(extractor.extract(texts)))

## 모델 예측의 정밀도 측정!
accuracy = accuracy_score(answer, preds)
print(f'\naccuracy : {accuracy*100:.2f}%')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


  0%|          | 0/712 [00:00<?, ?it/s]

  0%|          | 0/179 [00:00<?, ?it/s]


accuracy : 79.89%


### 5. csv로 저장 + kaggle에 submit

In [ ]:
import csv

ids = test['PassengerId'].values
texts = [", ".join(str(_)) for _ in test.iloc[:, 1:].values]
preds = classifier.predict(scaler.transform(extractor.extract(texts)))

with open('submission.csv', "w") as to_file:
    csvwriter = csv.writer(to_file, delimiter=",", quotechar='"')
    csvwriter.writerow(["PassengerId", "Survived"])
    for id, pred in zip(ids, preds):
        csvwriter.writerow([id, pred])

  0%|          | 0/418 [00:00<?, ?it/s]

In [ ]:
sub = pd.read_csv('submission.csv')
sub.head(5)

,PassengerId,Survived
0,892,1
1,893,1
2,894,1
3,895,0
4,896,1
